In [1]:
# db_elec_path = "c:\\Users\\mikel.perez\\0_Python_Projects\\shared_data\\db\\facturas_v1.db"
# cliente = "Montfrisa"
# tarifa = "6.1 TD"
# year = 2024
# cups = "ES0173000000400220DS"
# month = 12
# matrix_path = "c:\\Users\\mikel.perez\\0_Python_Projects\\verification_project\\data\\elec\\matrix_ver.pkl"
# path_datos_cliente = "c:\\Users\\mikel.perez\\0_Python_Projects\\verification_project/customers/elec/Montfrisa/datos_ELEC_Montfrisa_6.1 TD.xlsm"
# path_load_cliente = "c:\\Users\\mikel.perez\\0_Python_Projects\\verification_project/customers/elec/Montfrisa/load/2024/09. Septiembre"
# path_peajes_cargos = "c:\\Users\\mikel.perez\\0_Python_Projects\\shared_data\\data\\elec\\peajes_cargos.xlsx"
# path_ie = "c:\\Users\\mikel.perez\\0_Python_Projects\\shared_data\\data\\elec/ie.xlsx"
# path_final_matrix = "c:\\Users\\mikel.perez\\0_Python_Projects\\verification_project\\data\\elec\\final_matrix.pkl"
# df_coberturas_path = "c:\\Users\\mikel.perez\\0_Python_Projects\\verification_project\\data\\elec\\df_coberturas.pkl"

In [2]:
# Parameters
year = 2024
month = 12
ssaa_path = "c:\\Users\\mikel.perez\\0_Python_Projects - copia\\0_Python_projects_v2\\data\\processed\\elec/ssaa"
ssaa_path_df = "c:\\Users\\mikel.perez\\0_Python_Projects - copia\\0_Python_projects_v2\\data\\processed\\elec/ssaa.pkl"
perd_path = "c:\\Users\\mikel.perez\\0_Python_Projects - copia\\0_Python_projects_v2\\data\\processed\\elec/perdidas122024.pkl"
spot_path = "c:\\Users\\mikel.perez\\0_Python_Projects - copia\\0_Python_projects_v2\\data\\processed\\elec\\spot_elec.parquet"
spot_formated_path = "c:\\Users\\mikel.perez\\0_Python_Projects - copia\\0_Python_projects_v2\\data\\processed\\elec/spot_h_periodos.pkl"
futures_path = "c:\\Users\\mikel.perez\\0_Python_Projects - copia\\0_Python_projects_v2\\data\\processed\\elec\\futuros_elec.parquet"
path_omie_matrix = "c:\\Users\\mikel.perez\\0_Python_Projects - copia\\0_Python_projects_v2\\data\\customers\\elec\\PR2\\matrix_ver.pkl"
compodem_a2_path = "c:\\Users\\mikel.perez\\0_Python_Projects - copia\\0_Python_projects_v2\\data\\verification_project\\elec\\compodem_a2"

In [4]:
import os
import pandas as pd
from datetime import datetime, date

In [5]:
list = os.listdir(compodem_a2_path)

In [6]:
print(f"Found {len(list)} files in {compodem_a2_path}.")
# Initialize an empty DataFrame for concatenation
df_ssaa = pd.DataFrame()

# Rename `list` to a more descriptive variable name like `file_list`

for file_name in list:
    if 'compodem' in file_name:

        # Read the CSV file, skipping rows and setting no header initially
        df = pd.read_csv(f'{compodem_a2_path}/{file_name}', sep=';', skiprows=2, header=None)
        
        # Filter based on conditions in columns 2 and 3
        df = df[(df[3] == 'NOCUR') 
                # & 
                # (df[2].isin(['RT3', 'CT3', 'RT6', 'BS3', 'BALX', 'EXD', 'DSV', 'IN7', 'CFP', 'MAJ3', 'RAD3', 'RAD1']))
                ]
        
        # Group by the first two columns and sum the rest
        df_grouped = df.groupby([0, 1]).sum()
        
        # Drop unnecessary columns and reset index
        df_grouped = df_grouped.drop([2, 3], axis=1).reset_index()
        
        # Convert the date column to datetime
        df_grouped[0] = pd.to_datetime(df_grouped[0], format='%d/%m/%Y')
        
        # Define function to create `datetime` with custom hour logic
        def calculate_datetime(row, length):
            day, hour = row[0], int(row[1])
            if length == 25:
                hour_adjusted = hour - 1 if hour <= 3 else hour - 2
            elif length == 23:
                hour_adjusted = hour - 1 if hour <= 2 else hour
            else:
                hour_adjusted = hour - 1
            return datetime(day.year, day.month, day.day, hour_adjusted, 0)
        
        # Apply datetime transformation based on row count
        df_grouped['datetime'] = df_grouped.apply(lambda row: calculate_datetime(row, len(df_grouped)), axis=1)
        
        # Select and rename columns
        df_final = df_grouped[['datetime', 6]].copy()
        df_final.columns = ['datetime', 'ssaa']
        
        # Append to the main DataFrame
        df_ssaa = pd.concat([df_ssaa, df_final], ignore_index=True)

# Post-processing outside the loop for efficiency
df_ssaa = df_ssaa.sort_values(by='datetime').reset_index(drop=True)

# Extract month and year from datetime for each row
df_ssaa['month'] = df_ssaa['datetime'].dt.month
df_ssaa['year'] = df_ssaa['datetime'].dt.year

# Select the final column order
df_ssaa = df_ssaa[['datetime', 'month', 'year', 'ssaa']]

# Resulting DataFrame `df_concat` now holds the consolidated and processed data
print(f"SSAA DataFrame created with {len(df_ssaa)} rows.")


                datetime  month  year       ssaa
0    2024-12-01 00:00:00     12  2024   8.075517
1    2024-12-01 01:00:00     12  2024  10.680666
2    2024-12-01 02:00:00     12  2024  11.079547
3    2024-12-01 03:00:00     12  2024  12.813824
4    2024-12-01 04:00:00     12  2024  13.072237
...                  ...    ...   ...        ...
4986 2025-06-29 19:00:00      6  2025   6.969616
4987 2025-06-29 20:00:00      6  2025   7.866240
4988 2025-06-29 21:00:00      6  2025  13.033687
4989 2025-06-29 22:00:00      6  2025  11.451917
4990 2025-06-29 23:00:00      6  2025   4.545184

[4991 rows x 4 columns]


In [9]:
df_ssaa_fil = df_ssaa[(df_ssaa['year'] == year) & (df_ssaa['month'] == month)]
df_ssaa_fil = df_ssaa_fil.reset_index(drop=True)
df_ssaa_fil = df_ssaa_fil.reset_index(drop=False)
df_ssaa_fil = df_ssaa_fil[['index', 'ssaa']]
df_ssaa_fil

,index,ssaa
0,0,15.446519
1,1,21.972819
2,2,22.969168
3,3,23.856939
4,4,27.244467
...,...,...
691,691,6.969616
692,692,7.866240
693,693,13.033687
694,694,11.451917


In [10]:
# Append last 24 rows to ssaa dataframe in case last day of mont is missing

df_append = df_ssaa_fil.iloc[-24:]
df_ssaa_fil_ap = pd.concat([df_ssaa_fil, df_append], ignore_index=True)
df_ssaa_fil_ap.drop('index', axis=1, inplace=True)
df_ssaa_fil_ap.reset_index(drop=False, inplace=True)
df_ssaa_fil_ap

df_ssaa_fil = df_ssaa_fil_ap.copy()

df_ssaa_fil

,index,ssaa
0,0,15.446519
1,1,21.972819
2,2,22.969168
3,3,23.856939
4,4,27.244467
...,...,...
715,715,6.969616
716,716,7.866240
717,717,13.033687
718,718,11.451917


In [12]:
# Extract the pool data for month and year of estimation
df_pool = pd.read_pickle(spot_formated_path)
df_pool_fil = df_pool[(df_pool['year'] == year) & (df_pool['month'] == month)]
df_pool_fil = df_pool_fil.reset_index(drop=True)
df_pool_fil = df_pool_fil.reset_index(drop=False)

,index,datetime,value,date,month,year,time,periodo
0,0,2025-06-01 00:00:00,60.00,2025-06-01,6,2025,00:00:00,6
1,1,2025-06-01 01:00:00,36.07,2025-06-01,6,2025,01:00:00,6
2,2,2025-06-01 02:00:00,35.01,2025-06-01,6,2025,02:00:00,6
3,3,2025-06-01 03:00:00,34.95,2025-06-01,6,2025,03:00:00,6
4,4,2025-06-01 04:00:00,34.43,2025-06-01,6,2025,04:00:00,6
...,...,...,...,...,...,...,...,...
715,715,2025-06-30 19:00:00,111.38,2025-06-30,6,2025,19:00:00,3
716,716,2025-06-30 20:00:00,135.00,2025-06-30,6,2025,20:00:00,3
717,717,2025-06-30 21:00:00,175.01,2025-06-30,6,2025,21:00:00,3
718,718,2025-06-30 22:00:00,157.00,2025-06-30,6,2025,22:00:00,4


In [13]:
df_matrix = pd.merge(df_pool_fil, df_ssaa_fil, on='index')
df_matrix['perd'] = 0.07
df_matrix.drop('index', axis=1, inplace=True)
df_matrix = df_matrix[['datetime', 'periodo', 'value', 'ssaa', 'perd']]
print("Spot and ssaa data merged successfully.")


,datetime,periodo,value,ssaa,perd
0,2025-06-01 00:00:00,6,60.00,15.446519,0.07
1,2025-06-01 01:00:00,6,36.07,21.972819,0.07
2,2025-06-01 02:00:00,6,35.01,22.969168,0.07
3,2025-06-01 03:00:00,6,34.95,23.856939,0.07
4,2025-06-01 04:00:00,6,34.43,27.244467,0.07
...,...,...,...,...,...
715,2025-06-30 19:00:00,3,111.38,6.969616,0.07
716,2025-06-30 20:00:00,3,135.00,7.866240,0.07
717,2025-06-30 21:00:00,3,175.01,13.033687,0.07
718,2025-06-30 22:00:00,4,157.00,11.451917,0.07


In [14]:
df_matrix.to_pickle(path_omie_matrix)
print(f"Omie matrix saved to {path_omie_matrix}")